In [1]:
import pandas as pd
import numpy as np

np.__version__

'1.19.5'

# ⚠️ BE ADVISED: this notebook is a bit slow, but thanksfully doesn't seem to crash or overheat the computer ⚠️ 

# Pull in data (prepped in other notebook)

In [2]:
lyric_lines_ready_for_analysis = pd.read_csv('./data/3 OUTPUT One lyric lines per row, cleaned.csv')

In [3]:
lyric_lines_ready_for_analysis

,song_id,performer,song,generic_genre,lyric_line,chart_debut,chart_debut_date_as_a_decimal
0,#9 DreamJohn Lennon,John Lennon,#9 Dream,ROCK,So long ago,1974-12-21,1974.97
1,#9 DreamJohn Lennon,John Lennon,#9 Dream,ROCK,"Was it in a dream, was it just a dream?",1974-12-21,1974.97
2,#9 DreamJohn Lennon,John Lennon,#9 Dream,ROCK,"I know, yes I know",1974-12-21,1974.97
3,#9 DreamJohn Lennon,John Lennon,#9 Dream,ROCK,"Seemed so very real, it seemed so real to me",1974-12-21,1974.97
4,#9 DreamJohn Lennon,John Lennon,#9 Dream,ROCK,Took a walk down the street,1974-12-21,1974.97
...,...,...,...,...,...,...,...
166861,pride.is.the.devilJ. Cole & Lil Baby,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"I'm addicted to promethazine, it's crazy, yeah...",2021-05-29,2021.41
166862,pride.is.the.devilJ. Cole & Lil Baby,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"All this money coming in, it drive me crazy no...",2021-05-29,2021.41
166863,pride.is.the.devilJ. Cole & Lil Baby,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,I'll be crazy if I blow it,2021-05-29,2021.41
166864,pride.is.the.devilJ. Cole & Lil Baby,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,And it left so many R.I.P.,2021-05-29,2021.41


# EMBED WORDS USING Universal Encoder

In [4]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [5]:
model = hub.load('./data/universal-sentence-encoder-large_5/')

2022-12-22 01:10:16.720856: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def embed(input):
    return model(input)

## Ok, so the notebook crashes trying to embed *all at once*, so let's break this up:

In [7]:
len(lyric_lines_ready_for_analysis)

166866

In [8]:
lyrics_array = list(lyric_lines_ready_for_analysis.lyric_line.values)
# FOR TESTING:
lyrics_array = lyrics_array#[:20000]

In [9]:
def process_it(batch):
    return embed(batch).numpy()

def process_one_batch(batch, running_total):
    processed_batch = process_it(batch)
    return [*running_total, *processed_batch]

def batch_it(full_array, increment):
    running_total = []
    
    _max = len(full_array)
    SAFETY = 0

    lower_bound = 0
    upper_bound = lower_bound + increment

    while upper_bound < _max and SAFETY < _max:
        running_total = process_one_batch(full_array[lower_bound:upper_bound], running_total)
        lower_bound = upper_bound
        upper_bound = lower_bound + increment
        SAFETY = SAFETY + 1
        print(f'Finished: [{lower_bound} : {upper_bound}]')

    running_total = process_one_batch(full_array[lower_bound:_max], running_total)
    return running_total

res = batch_it(lyrics_array, increment=1000)
len(res)

Finished: [1000 : 2000]
Finished: [2000 : 3000]
Finished: [3000 : 4000]
Finished: [4000 : 5000]
Finished: [5000 : 6000]
Finished: [6000 : 7000]
Finished: [7000 : 8000]
Finished: [8000 : 9000]
Finished: [9000 : 10000]
Finished: [10000 : 11000]
Finished: [11000 : 12000]
Finished: [12000 : 13000]
Finished: [13000 : 14000]
Finished: [14000 : 15000]
Finished: [15000 : 16000]
Finished: [16000 : 17000]
Finished: [17000 : 18000]
Finished: [18000 : 19000]
Finished: [19000 : 20000]
Finished: [20000 : 21000]
Finished: [21000 : 22000]
Finished: [22000 : 23000]
Finished: [23000 : 24000]
Finished: [24000 : 25000]
Finished: [25000 : 26000]
Finished: [26000 : 27000]
Finished: [27000 : 28000]
Finished: [28000 : 29000]
Finished: [29000 : 30000]
Finished: [30000 : 31000]
Finished: [31000 : 32000]
Finished: [32000 : 33000]
Finished: [33000 : 34000]
Finished: [34000 : 35000]
Finished: [35000 : 36000]
Finished: [36000 : 37000]
Finished: [37000 : 38000]
Finished: [38000 : 39000]
Finished: [39000 : 40000]
Fin

166866

In [10]:
message_embeddings = res

In [11]:
message_embeddings_df = pd.DataFrame(message_embeddings)
message_embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.002448,-0.006144,-0.040928,-0.015099,0.000150,0.006566,0.039014,0.031581,0.039477,-0.020835,...,0.068633,0.010269,-0.014108,0.013764,-0.003254,-0.022148,-0.012551,-0.069721,-0.024095,-0.051011
1,-0.020579,0.016060,0.041853,0.015500,-0.010827,-0.013930,0.013164,-0.047087,0.033838,0.041109,...,-0.047296,0.016397,-0.076041,-0.024282,-0.036025,-0.027170,-0.038159,0.055621,0.067790,0.034361
2,-0.015094,-0.006433,0.014353,-0.013223,-0.046794,0.040840,-0.015416,-0.037821,-0.030475,-0.007043,...,0.032886,-0.052762,-0.024700,-0.091697,0.052754,-0.067762,0.012295,-0.058588,0.047908,-0.021352
3,-0.117747,-0.041903,-0.012537,0.030007,-0.015900,-0.003041,-0.067819,-0.047314,0.007004,0.041440,...,-0.030527,0.026838,-0.034646,0.008963,0.049549,-0.044417,-0.057669,0.043328,0.049226,0.008116
4,0.032745,-0.043692,0.053045,0.050328,-0.010547,0.006445,-0.019568,-0.074121,0.074195,-0.044683,...,-0.008463,-0.026073,-0.025741,-0.008665,-0.026599,-0.026849,0.021338,-0.069308,-0.044292,-0.008158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166861,0.069946,0.025872,-0.052863,0.105396,-0.031138,0.026127,0.038499,0.002888,-0.034377,-0.049539,...,0.039111,0.001513,-0.010167,-0.021516,0.014628,-0.089937,0.006847,0.057478,0.000188,-0.020785
166862,-0.055095,0.020369,-0.016676,0.018666,0.051288,0.071585,0.018204,0.045787,0.038067,-0.045270,...,0.073949,0.081896,-0.001522,0.032019,-0.053496,-0.053315,-0.042497,0.030564,0.039444,0.032298
166863,-0.019677,0.010472,-0.024140,-0.001454,0.023227,-0.020215,-0.020753,0.056700,0.050644,-0.082072,...,0.108221,0.085725,-0.020897,-0.051127,-0.054858,0.031241,-0.046810,0.014357,-0.029354,-0.021072
166864,0.033893,0.008252,-0.014001,-0.012160,0.014208,0.053781,0.056732,0.002518,-0.053152,0.016452,...,0.086646,-0.028210,-0.063976,0.092406,0.012750,0.014385,-0.009993,-0.025804,0.011054,0.015994


# Export!

In [12]:
message_embeddings_df.to_csv('./data/4 OUTPUT 1d embeddings array.csv', index=False)